In [1]:
import requests
import json
from time import time

# Helper functions to get data from the PokeAPI
def get_pokemon_by_name(name):
    url = f"https://pokeapi.co/api/v2/pokemon/{str(name).lower()}/"
    response = requests.get(url)
    return response.json() if response.status_code == 200 else None

def get_evolution_chain(pokemon_id):
    url = f"https://pokeapi.co/api/v2/pokemon-species/{pokemon_id}/"
    response = requests.get(url)
    if response.status_code == 200:
        species_data = response.json()
        evolution_url = species_data['evolution_chain']['url']
        evolution_response = requests.get(evolution_url)
        return evolution_response.json() if evolution_response.status_code == 200 else None
    return None

def get_generation(pokemon_id):
    url = f"https://pokeapi.co/api/v2/pokemon/{pokemon_id}/"
    response = requests.get(url)
    return response.json().get('generation', {}).get('name', None)

def get_habitat(pokemon_id):
    url = f"https://pokeapi.co/api/v2/pokemon-species/{pokemon_id}/"
    response = requests.get(url)
    return response.json().get('habitat', {}).get('name', None)

def get_shape(pokemon_id):
    url = f"https://pokeapi.co/api/v2/pokemon/{pokemon_id}/"
    response = requests.get(url)
    return response.json().get('shape', {}).get('name', None)

def get_color(pokemon_id):
    url = f"https://pokeapi.co/api/v2/pokemon/{pokemon_id}/"
    response = requests.get(url)
    return response.json().get('color', {}).get('name', None)

def get_types(pokemon_id):
    url = f"https://pokeapi.co/api/v2/pokemon/{pokemon_id}/"
    response = requests.get(url)
    if response.status_code == 200:
        return [type_info['type']['name'] for type_info in response.json()['types']]
    return []

# Rank calculation function
def calculate_pokemon_rank(guessed_pokemon, target_pokemon):
    total_score = 0

    # Evolution chain
    guessed_evolution = get_evolution_chain(guessed_pokemon['id'])
    target_evolution = get_evolution_chain(target_pokemon['id'])

    if guessed_evolution and target_evolution:
        if guessed_evolution['chain']['species']['name'] == target_evolution['chain']['species']['name']:
            total_score += 10
        else:
            total_score += 1

    # Type Similarity
    guessed_types = get_types(guessed_pokemon['id'])
    target_types = get_types(target_pokemon['id'])
    for type in guessed_types:
        if type in target_types:
            total_score += 5

    # Generation
    guessed_generation = get_generation(guessed_pokemon['id'])
    target_generation = get_generation(target_pokemon['id'])
    if guessed_generation == target_generation:
        total_score += 5
    elif abs(int(guessed_generation) - int(target_generation)) == 1:
        total_score += 3
    else:
        total_score += 1

    # Habitat
    guessed_habitat = get_habitat(guessed_pokemon['id'])
    target_habitat = get_habitat(target_pokemon['id'])
    if guessed_habitat == target_habitat:
        total_score += 5

    # Shape
    guessed_shape = get_shape(guessed_pokemon['id'])
    target_shape = get_shape(target_pokemon['id'])
    if guessed_shape == target_shape:
        total_score += 5

    # Color
    guessed_color = get_color(guessed_pokemon['id'])
    target_color = get_color(target_pokemon['id'])
    if guessed_color == target_color:
        total_score += 3

    return {'name': guessed_pokemon['name'], 'score': total_score}

def get_all_pokemon_rank(target_pokemon):
    all_pokemons = []
    for i in range(1, 1010):  # Change the limit to 1010 later
        pokemon = get_pokemon_by_name(i)
        if pokemon:
            rank = calculate_pokemon_rank(pokemon, target_pokemon)
            all_pokemons.append(rank)

    all_pokemons.sort(key=lambda x: x['score'], reverse=True)
    return all_pokemons

def precompute_ranks():
    all_ranks = {}

    # Loop through first 20 Pokémon for testing (can be adjusted later)
    for i in range(1, 1010):
        print(f"Processing Pokémon ID: {i}")
        target_pokemon = get_pokemon_by_name(i)
        if target_pokemon:
            ranks = get_all_pokemon_rank(target_pokemon)
            all_ranks[target_pokemon['name']] = ranks

    # Save the precomputed data into a JSON file
    with open('pokemon_ranks.json', 'w') as file:
        json.dump(all_ranks, file, indent=2)
    print("Precomputed ranks saved to pokemon_ranks.json")

if __name__ == "__main__":
    start_time = time()
    precompute_ranks()
    end_time = time()
    print(f"Precomputing completed in {end_time - start_time:.2f} seconds")


Processing Pokémon ID: 1


AttributeError: 'NoneType' object has no attribute 'get'

In [ ]:
import requests
import json
import concurrent.futures

# Function to get Pokémon data from PokeAPI
def get_pokemon_data(pokemon_id):
    url = f"https://pokeapi.co/api/v2/pokemon/{pokemon_id}/"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    return None

def get_evolution_chain(pokemon_id):
    url = f"https://pokeapi.co/api/v2/pokemon-species/{pokemon_id}/"
    response = requests.get(url)
    if response.status_code == 200:
        species_data = response.json()
        evolution_url = species_data.get('evolution_chain', {}).get('url', None)
        if evolution_url:
            evo_response = requests.get(evolution_url)
            if evo_response.status_code == 200:
                evo_data = evo_response.json()
                return evo_data
            else:
                print(f"Failed to retrieve evolution chain for Pokémon ID: {pokemon_id}")  # Log failed request
                print(f"Evolution URL: {evolution_url}")
        else:
            print(f"No evolution chain URL found for Pokémon ID: {pokemon_id}")  # Log missing evolution chain URL
    else:
        print(f"Failed to retrieve species data for Pokémon ID: {pokemon_id}")  # Log failed API request
    return None

def get_generation(pokemon_id):
    url = f"https://pokeapi.co/api/v2/pokemon-species/{pokemon_id}/"
    response = requests.get(url)
    if response.status_code == 200:
        species_data = response.json()
        generation_data = species_data.get('generation', None)
        if generation_data:
            return generation_data.get('name')  # Return the generation name as a string
        else:
            print(f"Generation data missing for Pokémon ID: {pokemon_id}")  # Log the missing generation data
            print(f"Response: {species_data}")  # Log the full response for debugging
    else:
        print(f"Failed to retrieve species data for Pokémon ID: {pokemon_id}")  # Log failed API request
    return None

def get_habitat(pokemon_id):
    url = f"https://pokeapi.co/api/v2/pokemon-species/{pokemon_id}/"
    response = requests.get(url)
    if response.status_code == 200:
        species_data = response.json()
        return species_data.get('habitat', {}).get('name', '')
    return None

def get_shape(pokemon_id):
    url = f"https://pokeapi.co/api/v2/pokemon/{pokemon_id}/"
    response = requests.get(url)
    if response.status_code == 200:
        pokemon_data = response.json()
        return pokemon_data.get('shape', {}).get('name', '')
    return None

def get_color(pokemon_id):
    url = f"https://pokeapi.co/api/v2/pokemon-species/{pokemon_id}/"
    response = requests.get(url)
    if response.status_code == 200:
        species_data = response.json()
        return species_data.get('color', {}).get('name', '')
    return None

def calculate_rank(target_pokemon_data, pokemon_id):
    pokemon_data = get_pokemon_data(pokemon_id)
    if not pokemon_data:
        return None

    total_score = 0

    # 1. Evolution chain
    guessed_evolution = get_evolution_chain(pokemon_id)
    target_evolution = get_evolution_chain(target_pokemon_data['id'])
    if guessed_evolution and target_evolution:
        if guessed_evolution['chain']['species']['name'] == target_evolution['chain']['species']['name']:
            total_score += 10
        else:
            total_score += 1
    else:
        print(f"Skipping Pokémon ID {pokemon_id} due to missing evolution data.")  # Log the skipped Pokémon

    # 2. Type similarity
    guessed_types = {t['type']['name'] for t in pokemon_data['types']}
    target_types = {t['type']['name'] for t in target_pokemon_data['types']}
    common_types = guessed_types & target_types
    total_score += len(common_types) * 5

    # 3. Generation comparison (direct string comparison)
    guessed_generation = get_generation(pokemon_id)
    target_generation = get_generation(target_pokemon_data['id'])
    if guessed_generation == target_generation:
        total_score += 5
    elif guessed_generation and target_generation:
        # Allow for slight mismatches, but not too far apart
        if abs(ord(guessed_generation[-1]) - ord(target_generation[-1])) <= 1:
            total_score += 3
        else:
            total_score += 1
    else:
        print(f"Skipping Pokémon ID {pokemon_id} due to missing generation data.")  # Log the skipped Pokémon

    # 4. Habitat
    guessed_habitat = get_habitat(pokemon_id)
    target_habitat = get_habitat(target_pokemon_data['id'])
    if guessed_habitat == target_habitat:
        total_score += 5

    # 5. Shape
    guessed_shape = get_shape(pokemon_id)
    target_shape = get_shape(target_pokemon_data['id'])
    if guessed_shape == target_shape:
        total_score += 5

    # 6. Color
    guessed_color = get_color(pokemon_id)
    target_color = get_color(target_pokemon_data['id'])
    if guessed_color == target_color:
        total_score += 3

    return {"name": pokemon_data["name"], "score": total_score}

# Function to compute ranks in parallel using multithreading
def compute_ranks_in_parallel(target_pokemon_id, pokemon_range_start, pokemon_range_end):
    target_pokemon_data = get_pokemon_data(target_pokemon_id)
    if not target_pokemon_data:
        print(f"Failed to retrieve target Pokémon data for ID: {target_pokemon_id}")
        return

    results = []
    with concurrent.futures.ThreadPoolExecutor() as executor:
        # List all Pokémon IDs to process
        pokemon_ids = range(pokemon_range_start, pokemon_range_end + 1)

        # Map the rank calculation function to all Pokémon IDs
        future_to_pokemon = {executor.submit(calculate_rank, target_pokemon_data, pokemon_id): pokemon_id for pokemon_id in pokemon_ids}

        # Collect results as they complete
        for future in concurrent.futures.as_completed(future_to_pokemon):
            pokemon_id = future_to_pokemon[future]
            try:
                result = future.result()
                if result:
                    results.append(result)
            except Exception as e:
                print(f"Error processing Pokémon ID {pokemon_id}: {e}")

    return results

# Save the computed ranks to a JSON file
def save_to_json(data, filename="pokemon_ranks.json"):
    with open(filename, "w") as f:
        json.dump(data, f, indent=4)

# Main function to run the precomputation
def run_precomputation(target_pokemon_id, pokemon_range_start, pokemon_range_end):
    print(f"Precomputing ranks for target Pokémon ID: {target_pokemon_id} in the range {pokemon_range_start}-{pokemon_range_end}...")

    results = compute_ranks_in_parallel(target_pokemon_id, pokemon_range_start, pokemon_range_end)

    if results:
        # Sort the results by score in descending order (highest rank first)
        sorted_results = sorted(results, key=lambda x: x['score'], reverse=True)

        # Ensure the target Pokémon is at the top
        target_pokemon_name = get_pokemon_data(target_pokemon_id)['name']
        sorted_results.sort(key=lambda x: (x['name'] != target_pokemon_name, x['score']), reverse=True)

        # Save sorted results to JSON
        save_to_json(sorted_results)
        print(f"Precomputation completed. Results saved to 'pokemon_ranks.json'.")
    else:
        print("Precomputation failed. No data to save.")

if __name__ == "__main__":
    target_pokemon_id = 1  # Example target Pokémon (Bulbasaur)
    run_precomputation(target_pokemon_id, 1, 1010)  # Test with Pokémon IDs 1 to 1010


Precomputing ranks for target Pokémon ID: 1 in the range 1-1010...
Error processing Pokémon ID 388: 'NoneType' object has no attribute 'get'
Error processing Pokémon ID 387: 'NoneType' object has no attribute 'get'
Error processing Pokémon ID 391: 'NoneType' object has no attribute 'get'
Error processing Pokémon ID 390: 'NoneType' object has no attribute 'get'
Error processing Pokémon ID 389: 'NoneType' object has no attribute 'get'
Error processing Pokémon ID 392: 'NoneType' object has no attribute 'get'
Error processing Pokémon ID 393: 'NoneType' object has no attribute 'get'
Error processing Pokémon ID 399: 'NoneType' object has no attribute 'get'
Error processing Pokémon ID 400: 'NoneType' object has no attribute 'get'
Error processing Pokémon ID 396: 'NoneType' object has no attribute 'get'
Error processing Pokémon ID 395: 'NoneType' object has no attribute 'get'
Error processing Pokémon ID 394: 'NoneType' object has no attribute 'get'
Error processing Pokémon ID 397: 'NoneType' o